In [1]:
from QLBM import QLBM_2ndCL
from qiskit import transpile
import qiskit_aer
import numpy as np
import matplotlib.pyplot as plt

In [2]:
Nq = 9  # Number of lattice directions
Nx = 4  # Number of grid points in x-direction
Ny = 4  # Number of grid points in y-direction
Nt = 3  # Number of timesteps

# Lattice vector definition
ei = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8])
ci = np.array([[0, 1, 0, -1, 0, 1, -1, -1, 1], 
               [0, 0, 1, 0, -1, 1, 1, -1, -1]])

# Lattice weights definition
omega1, omega2, omega3 = 1.0, 1.5, 1.9
weights = np.array([4/9, 1/9, 1/9, 1/9, 1/9, 1/36, 1/36, 1/36, 1/36])

In [3]:
# Meshgrid definition in 2D XY domain
t = np.arange(Nt)
xm = np.arange(Nx)
ym = np.arange(Ny)
X, Y = np.meshgrid(xm, ym, indexing='ij')

# fi field initialization
A_x = 0.3
k_x = 1

fi_ini = np.zeros((Nx, Ny, Nq))
for i, wi in zip(ei, weights):
    # fi_ini = fi_ini.at[X, Y, i].set(wi * (1 + A_x * np.cos(2 * np.pi * k_x * Y / Ny) * ci[0][i]))
    fi_ini[X, Y, i] = wi * (1 + A_x * np.cos(2 * np.pi * k_x * Y / Ny) * ci[0][i])

In [4]:
simulator = qiskit_aer.backends.statevector_simulator.StatevectorSimulator()

In [5]:
ci_cj = np.tensordot(ci[0], ci[0], axes=0) + np.tensordot(ci[1], ci[1], axes=0) # ci is equivalent to cj
L_ij = np.transpose(weights * (1 + 3 * ci_cj))
A_ij = (1 - omega1) * np.eye(Nq) + omega1 * L_ij

In [6]:
# Create and run the quantum circuit for LBM
qc = QLBM_2ndCL(fi_field=fi_ini, CL_matrix=A_ij, number_velocities=Nq, timesteps=1)

compiled_circuit = transpile(qc, simulator)
result = simulator.run(compiled_circuit).result()

# Process the quantum statevector to update Psi_qlbm
statevector = np.array(result.get_statevector())
real_part_statevector = np.real(statevector[:Nx * Ny * Nq])
real_part_statevector_reshaped = np.reshape(real_part_statevector, (Nx, Ny, Nq))

# Normalize and update the scalar field for the next timestep
fi_final = real_part_statevector_reshaped

2.29128784747792
Current iteration circuit: 0


In [7]:
def get_macro_quantities(fi):
    rho = np.sum(fi, axis=2)  # Density
    ux = np.sum(fi * ci[0], 2) / rho
    uy = np.sum(fi * ci[1], 2) / rho
    
    return rho, ux, uy

In [8]:
rho, ux, uy = get_macro_quantities(2.29128784747792 * fi_final)
rho, ux, uy

(array([[ 1.27494162e+00,  8.36106784e-01,  5.29926835e-02,
         -3.11721668e-02],
        [ 1.04946295e-01,  2.53533623e-01, -1.03907223e-03,
          1.24688667e-02],
        [ 3.20034245e-01,  5.09145390e-02,  1.96384651e-01,
          1.33001245e-01],
        [ 1.45470112e-02,  1.08063511e-01,  8.95680258e-01,
          8.18788913e-01]]),
 array([[-4.88997555e-03, -6.21375311e-03,  3.72549020e-01,
         -3.90000000e+00],
        [-1.28712871e-01, -2.62295082e-01,  1.60000000e+01,
          2.25000000e+00],
        [ 3.73376623e-01,  3.08163265e+00, -4.60317460e-01,
         -1.02343750e+00],
        [-2.07142857e+00,  1.70192308e+00,  4.75638051e-02,
          1.72588832e-01]]),
 array([[ 3.25998370e-03,  1.43330572e-01,  5.68627451e-01,
         -2.23333333e+00],
        [ 7.92079208e-02, -5.45081967e-01,  1.50000000e+02,
          9.58333333e+00],
        [-9.74025974e-03, -2.06122449e+00,  1.55026455e+00,
         -7.03125000e-02],
        [-7.28571429e+00,  3.36538462e-

In [9]:
qc.draw()

»
qx_0: »
      »
qx_1: »
      »
qy_0: »
      »
qy_1: »
      »
qf_0: »
      »
qf_1: »
      »
qf_2: »
      »
qf_3: »
      »
 anc: »
      »
«      ┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐»
«qx_0: ┤0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             ├»
«      │                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    